# Recherche d'information : librairie PyTerrier

Dans cette partie, nous nous intéressons à la librairie [PyTerrier](https://pyterrier.readthedocs.io/en/latest/#) qui permet de mettre en place diverses briques d'un moteur de recherche.
PyTerrier est basée sur [Terrier](http://terrier.org/) qui est un moteur de recherche développé en Java.

Nous allons voir : 
*   l'installation et la configuration
*   l'indexation d'une collection
*   l'accès à l'index
*   l'évaluation d'un moteur de recherche


## Installation ete configuration

Après l'installation de la librairie, il est nécessaire d'initialiser PyTerrier pour importer les fichiers jar et démarrer la machine virtuelle associée; 

In [1]:
!pip install python-terrier

import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 34.1 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 1.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 612.1 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 1.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.5/311.5 kB 1.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 107.4 kB/s eta 0:00:0000:0100:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 418.5 kB/s eta 0:00:

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


## Indexation d'une collection

Il est possible d'indexer plusieurs formats de collection : format TREC, fichiers en texte brut ou en PDF, ou encore des Dataframe Pandas ([pour plus de détails](https://pyterrier.readthedocs.io/en/latest/terrier-indexing.html)).

Un petit exemple à titre illustratif est foourni dans le code suivant :

In [2]:
import pandas as pd

# configuration de l'affichage
pd.set_option('display.max_colwidth', 150)

# le jeu de données au format DataFrame
docs_df = pd.DataFrame([
        ["d1", "this is the first document of many documents"],
        ["d2", "this is another document"],
        ["d3", "the topic of this document is unknown"]
    ], columns=["docno", "text"])

# création de l'index
indexer = pt.DFIndexer("./index_3docs", overwrite=True)         # Définition du format de données (DFIndexer())
index_ref = indexer.index(docs_df["text"], docs_df["docno"])    # Indexation des champs text et docno
!ls -lh index_3docs/                                            # Affichage de l'index sauvegardé dans "./index_3docs/"

total 36K
-rw-r--r-- 1 root root    3 Jun 17 11:51 data.direct.bf
-rw-r--r-- 1 root root   51 Jun 17 11:51 data.document.fsarrayfile
-rw-r--r-- 1 root root    4 Jun 17 11:51 data.inverted.bf
-rw-r--r-- 1 root root  344 Jun 17 11:51 data.lexicon.fsomapfile
-rw-r--r-- 1 root root  249 Jun 17 11:51 data.lexicon.fsomaphash
-rw-r--r-- 1 root root   24 Jun 17 11:51 data.meta.idx
-rw-r--r-- 1 root root   48 Jun 17 11:51 data.meta.zdata
-rw-r--r-- 1 root root 4.1K Jun 17 11:51 data.properties


De nombreux fichiers sont créés : index direct, index inverse, méta-données de l'index et de la configuration de l'indexation, etc...


Il est également possible de modifier la configuration de l'indexation : [voir ici](https://pyterrier.readthedocs.io/en/latest/terrier-indexing.html#indexing-configuration).

Pour chager un index existant en local :

In [ ]:
index = pt.IndexFactory.of(index_ref)

Il est aussi possible de voir les statistiques de l'index. 
pour connaître toutes les fonctions d'interrogation, se référencer à la [javadoc](http://terrier.org/docs/current/javadoc/org/terrier/structures/Index.html).

In [ ]:
# statistiques de la collection
print(index.getCollectionStatistics().toString())

In [ ]:
# statistiques du vocabulaire.
# Nt : document frequency : nombre de documents contenant le terme
# TF : term frequency : nombre d'occurences du terme
# maxTF : nombre d'occurences maximal pour un document
for kv in index.getLexicon():
  print("%s (%s) -> %s (%s)" % (kv.getKey(), type(kv.getKey()), kv.getValue().toString(), type(kv.getValue()) ) )

In [ ]:
# focus sur un terme particulier
index.getLexicon()["document"].toString()

In [ ]:
# récupère les statistiques de l'index inverse à partir d'un terme particulier
pointer = index.getLexicon()["document"]
for posting in index.getInvertedIndex().getPostings(pointer):
    print(posting.toString() + " doclen=%d" % posting.getDocumentLength())


De plus, PyTerrier met à disposition [une collection de jeux de données pré-traités](https://pyterrier.readthedocs.io/en/latest/datasets.html).
Dans ce qui suit, nous allons nous concentrer sur le jeu de données CORD19 qui recense des articles liés à la crise sanitaire Covid-19. Il est

In [ ]:
import os

cord19 = pt.datasets.get_dataset('irds:cord19/trec-covid')
pt_index_path = './terrier_cord19'

if not os.path.exists(pt_index_path + "/data.properties"):
  # création de l'index. Utilisation de l'itérateur pour parcourir la collection
  indexer = pt.index.IterDictIndexer(pt_index_path)

  # on donne à l'index la fonction pour parcourir l'index avec l'itérateur  get_corpus_iter() 
  # On spécifie les champs à indexer et les meta-données à sauvegarder
  index_ref = indexer.index(cord19.get_corpus_iter(), 
                            fields=('abstract',), 
                            meta=('docno',))

else:
  # dans le cas où l'index existe déjà
  index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")
index = pt.IndexFactory.of(index_ref)

**Exercice 1**

Afficher les statistiques de l'index Cord19 et analyser statistiques du terme "tv" (pas trop fréquent pour question d'affichage).

## Recherche de documents à partir de l'index

Pour effectuer une recherche dans l'index, il faut utiliser la fonction batchRetrieve qui prend en paramètre l'index et le modèle de pondération (tf, tf-idf, etc...). La liste des modèles supportés est disponible [ici](http://terrier.org/docs/current/javadoc/org/terrier/matching/models/package-summary.html).

In [ ]:
br = pt.BatchRetrieve(index, wmodel="Tf")
br.search("chemical reactions")

On récupère alors un DataFrame dont les colonnes sont les suivantes : 
*   qid : identifiant de la requête. Par défaut, il s'agit de "1", puisqu'il s'agit de notre première et unique requête.
*   docid : l'identifiant interne de Terrier pour chaque document
*   docno : l'identifiant unique externe (chaîne de caractères) pour chaque document
*   score : score des documents selon le modèle choisi (ici : fréquence totale des tf des termes de la requête dans chaque document)
*   rank : rang du document dénotant l'ordre décroissant par score.
*   query : la requête d'entrée

In [ ]:
# autre exemple de modèle : TF-IDF
tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF")
tfidf.search("chemical reactions")

On peut aussi fournir plusieurs requêtes grâce à un dataFrame. Pour interroger l'index, on applique la fonction transform() au BatchRetriever (br).
pour plus de détails, voir [les propriétés des transformations](https://pyterrier.readthedocs.io/en/latest/transformer.html) ainsi que les [opérations possibles](https://pyterrier.readthedocs.io/en/latest/operators.html).

In [ ]:
import pandas as pd
queries = pd.DataFrame([["q1", "document"], ["q2", "first document"]], columns=["qid", "query"])
br.transform(queries)       # ou aussi : br(queries)

**Exercice 2**

Ordonnancer les documents pour 3 requêtes : "covid disease", "hospital" et "home".
La fonction d'ordonnacement devra être de la forme suivante : 


```
0.4 * score_Bm25 + 0.6 * score_Dirichlet
```


**Excercice 3**

Créez un ordonnanceur qui effectue les opérations suivantes :
* obtient les 10 documents les mieux notés par fréquence de terme (wmodel="Tf")
* obtenir les 10 documents les mieux notés par TF.IDF (wmodel="TF_IDF")
* ré-ordonne uniquement les documents trouvés dans les DEUX paramètres de recherche précédents en utilisant BM25.

Combien de documents sont récupérés par ce pipeline complet pour la requête "chemical"?

Vérification : le document avec le docno "37771" devrait avoir un score de 12.426309 $ pour la requête "chemical".

## Reformulation de requêtes 

Il est également possible de mettre en place des pipelines de [reformulation de requêtes](https://pyterrier.readthedocs.io/en/latest/rewrite.html).

In [ ]:
bo1 = pt.rewrite.Bo1QueryExpansion(index)
dph = pt.BatchRetrieve(index, wmodel="DPH")
pipeline = dph >> bo1 >> dph
pipeline.search("chemical reactions")

L'autre solution est de l'intégrer directement dans la fonction d'ordonnancement. Mais la requête reformulée n'est pas visible et la solution précédente fait prendre conscience de la pipeline faite par le système de RI (ranking >> reformulation >> ranking quand on utilise des modèles basés sur la relevance feedback. ou reformulation >> ranking sinon). Plus d'exemples [ici](https://pyterrier.readthedocs.io/en/latest/rewrite.html).

In [ ]:
# modèle DPH avant reformulation de requête
pipelineQE = pt.BatchRetrieve(index, wmodel="DPH", controls={"qemodel" : "Bo1", "qe" : "off"})
pipelineQE.search("chemical reactions")

In [ ]:
# modèle DPH après reformulation de requête
pipelineQE = pt.BatchRetrieve(index, wmodel="DPH", controls={"qemodel" : "Bo1", "qe" : "on"})
pipelineQE.search("chemical reactions")

### Evaluation d'un système de recherche d'information

Pour évaluer un système de RI, il est nécessaire d'avoir un jeu de données constitué de requêtes et de jugements de pertinence. 

In [ ]:
# exemple de 5 requêtes pour cord19
cord19.get_topics(variant='title').head(5)

In [ ]:
# exemple de jugements de pertinence pour les 5 premières requêtes
cord19.get_qrels().head(5)

**Exercice 4**

A partir des requêtes et des jugements de pertinence du jeu de données CORD19, Ecrire le code qui permet d'afficher les résultats de la première requête de Cord19. L'affichage fusionnera les colonnes retournées par le BatchRetriever et les colonnes des qrels (merge sur qid et docno pour rajouter label et iteration au tableau). 

Il existe cependant une fonction qui permet de calculer l'efficacité de ces ordonnancements au travers des métriques d'évaluation (map, précision, rappel, ndcg, ...)

In [ ]:
pt.Experiment(
    [tfidf],
    cord19.get_topics(variant='title'),
    cord19.get_qrels(),
    eval_metrics=["map", "ndcg"])

,name,map,ndcg
0,BR(TF_IDF),0.180008,0.370795


**Exercice 5**

Réaliser une expérience comparant l'expansion de requêtes avec le modèle Bo1 et basé sur la KL-divergence; L'expérience est réalisée sur TREC CORD19 avec le modèle de référence BM25. Vous devrez construire des pipelines appropriées (plus de détails sur [l'expansion](https://pyterrier.readthedocs.io/en/latest/rewrite.html) et les [expérimentations](https://pyterrier.readthedocs.io/en/latest/experiments.html)).

Quelles approches entraînent des augmentations significatives de NDCG et MAP ou autres métriques ?